# Fitts' Law: A closer look

Many user interactions are very similar in nature. For example: Move the cursor to a button, slide a slider to a position, or turn a dial to a position.
All these have a starting point (where the cursor, the slider, or the dial were), and a region that is targeted (a button or the wanted position on the slider or dial).

Fitts' Law predicts the time that it will take to do these interactions, *without needing to test this in high-effort user studies*.

Run the code below to generate data that drives the message home: Try clicking the red circles repeatedly. Try this for N = 2 first to warm up, then change this to a higher number (e.g., N = 30) to generate more interesting data.

In [ ]:
####### run this code and then scroll down to see the resulting canvas #######
from IPython.display import HTML
import random

HTML("""
<canvas id="fitts" style="width:100%;height:400px;border:1px;"></canvas>
<p id="info">Click the target to begin.</p><script>
const canvas = document.getElementById("fitts");
canvas.width = canvas.clientWidth; canvas.height = canvas.clientHeight;
const ctx = canvas.getContext("2d");
const info = document.getElementById("info");
let target = {}; let lastTime = null; let lastx = null;
let times = []; let ws = []; let ds = []; let N = 20;

function newTarget() {
  target.size = Math.floor( Math.random()*120 + 10 );
  target.x = Math.random()*(canvas.clientWidth - target.size*2) + target.size;
  ctx.clearRect(0,0,canvas.clientWidth,400);
  ctx.beginPath(); ctx.arc(target.x, 200, target.size, 0, 2 * Math.PI);
  ctx.fillStyle = "red"; ctx.fill();
}

canvas.onclick = (e) => {
  const rect = canvas.getBoundingClientRect();
  const d = Math.hypot(e.clientX-rect.left-target.x,e.clientY-rect.top-200);
  if (d <= target.size) {
    const now = performance.now();
    if (lastTime) {
      times.push(Math.floor(now-lastTime));
      ws.push(target.size); ds.push( Math.floor(Math.abs(target.x-lastx)) );
      info.innerText = `Run ${times.length}/${N}`;
    }
    lastTime = now; lastx = target.x;
    if (times.length >= N) {
      info.innerText = ` times=[${times}]\n widths=[${ws}]\n dists=[${ds}]`;
      return;
    }
    newTarget();
  }
};
newTarget();
</script>
""")

Next, copy the resulting three lines at the end of the previous cell, in the code cell below and run the code. This will allow us to check the data here in the notebook:

In [ ]:
# Copy here the copied variables that were delivered from the cell above:
times=[832,1133,617,799,633,666,716,899,816,1165,750,733,916,716,950,699,733,851,549,600]
widths=[56,17,114,122,124,104,69,27,49,14,31,109,14,101,68,38,125,36,96,32]
dists=[520,12,119,757,61,385,363,804,656,392,495,424,872,490,182,60,221,665,18,43]

We first examine what the influence of the size (width) of the target circles was, on how quickly they were clicked. Then, we do the same for the distance to the circle from the last position. For both, we plot the data as scatter plot points:

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 2, figsize=(12,4))

axes[0].scatter(widths, times)
axes[0].set_xlabel("width (px)")
axes[0].set_ylabel("time (ms)")
axes[0].set_title("width vs time")

axes[1].scatter(dists, times)
axes[1].set_xlabel("distance (px)")
axes[1].set_title("distance vs time")

plt.tight_layout()
plt.show()

The above plots should show that there is not really a trend visible. You can have large distances to the target circle, but if that circle is huge, it can still be reached fast. The same for short distances and tiny circles.

That is why Fitts' Law combines both in the so-called Index of Difficulty$_1$:

$$
ID = \log_2 \left(1 + \frac{D}{W} \right)
$$

where $D$ is the distance to the target and $W$ is the width of the target

First, we visualize how the Index of Difficulty fares over time:

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

times = np.array(times)
IDs = np.array([math.log2(1 + d/w) for d, w in zip(dists, widths)])

b, a = np.polyfit(IDs, times, 1)  # Linear regression: T = a + b·ID

plt.figure()
plt.scatter(IDs, times)
plt.plot(IDs, a + b*IDs, 'r-')
plt.xlabel("Index of Difficulty (ID = log₂(1 + D/W))")
plt.ylabel("time (T) in ms")
plt.title("Index of Difficulty vs time")
plt.text(max(IDs), min(times), f"T ≈ {a:.1f} + {b:.1f}·ID", color='red',
         fontsize=12, ha='right')
plt.show()

The above plot should show a much clearer trend than the previous two plots. The red line is a linear regression fit over the data, which should show a close fit and a linear trend.

**The higher a circle's Index of Difficulty, the longer it took to click it.**

Combining the distance $D$ and width $W$ in this way ($D/W$) should be clear: The bigger the distance to the circle or the smaller its width, the longer it will take to click it. The Index of Difficulty uses this term in the $log_2$ function because it describes better the movement that is made. Plotting the function $log_2(1 + X)$ and the $ID$ for multiple widths and distances illustrates this:


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(2, 2, figsize=(16,8))

def ID(distance, width):
    """Calculate Fitts's Law's Index of Difficulty"""
    return np.log2(1 + distance / width)

x = np.linspace(1, 70, 70)
axes[0][0].plot(x, np.log2(1+x))
axes[0][0].set_xlabel("x")
axes[0][0].set_ylabel("log₂(1 + x)"); axes[0][0].set_title("log₂(1 + x)")

distances = np.linspace(1, 70, 10)
for width, color in zip([1, 3, 10], ['r','g','b']):
    axes[1][0].plot(distances, ID(distances, width), color+'.-', label=f'width = {width}')
axes[1][0].set_xlabel("distance (cm)")
axes[1][0].set_ylabel("log₂(1 + d/w)")
axes[1][0].set_title("distance vs Index of Difficulty")
axes[1][0].legend()

x = np.linspace(1, 10, 100)
axes[0][1].plot(x, np.log2(1+1/x))
axes[0][1].set_xlabel("x")
axes[0][1].set_ylabel("log₂(1 + 1/x)"); axes[0][1].set_title("log₂(1 + 1/x)")

widths = np.linspace(1, 10, 10)
for distance, color in zip([1, 20, 70], ['r','g','b']):
    axes[1][1].plot(widths, ID(distance, widths), color+'.-', label=f'dist = {distance}')
axes[1][1].set_xlabel("width (cm)")
axes[1][1].set_ylabel("log₂(1 + d/w)")
axes[1][1].set_title("width vs Index of Difficulty")
axes[1][1].legend();

plt.tight_layout(); plt.show()

The final addition to get to the formula of Fitts' Law is to add an offset $a$ and a scaling factor $b$ to the Index of Difficulty. With this, the formula for Fitts' Law is:

$$
T = a + b * \log_2 \left(1 + \frac{D}{W} \right)
$$

where $D$ is the distance to the target, $W$ is the width of the target, $a$ is a delay parameter (representing a constant time to start and stop the movement), and $b$ allows scaling the Index of Difficulty.

The parameters $a$ and $b$ are usually obtained through experiments (see the linear regression method here and note we already used $a$ and $b$ as such) for particular user interactions and interaction devices (such as a mouse pointer, a slide, or a dial).

The code in the next cell will plot the data from the circle-clicking experiment and allow $a$ and $b$ to be modified, showing their behavior.


In [ ]:
import ipywidgets as widgets
from IPython.display import display

ID_range = np.linspace(IDs.min(), IDs.max(), 100)
def plot_fitts(a, b):
    plt.figure()
    plt.scatter(IDs, times)
    plt.plot(ID_range, a + b*ID_range)
    plt.xlabel("Index of Difficulty (ID = log₂(1 + D/W))")
    plt.ylabel("Time (ms)")
    plt.title(f"Fitts' Law: T = {a:.0f} + {b:.0f}·ID")
    plt.show()

widgets.interact(
    plot_fitts,
    a=widgets.IntSlider(min=0, max=1000, step=20, value=a),
    b=widgets.IntSlider(min=0, max=500, step=10, value=b)
)

**Footnotes**

1. The original formula by Paul Fitts in 1954 used this term for the Index of Difficulty: $ID = \log_2 \left(2 * \frac{D}{W} \right)$ . You can change the above formulas in the code to this form, to verify the results are very similar.